In [0]:
%sql

SELECT DISTINCT SLOT_MNEMONIC FROM 4_prod.raw.mill_sch_appt
WHERE SLOT_MNEMONIC ILIKE '%antenatal%'
LIMIT 100

In [0]:
%sql

SELECT * FROM 3_lookup.mill.mill_code_value
WHERE DESCRIPTION LIKE '%CMV%'

In [0]:
%sql
-- Number of all adult women who received antenatal care between 2019-12-01 and 2024-12-31 
SELECT COUNT(DISTINCT a.PERSON_ID)
FROM 4_prod.raw.mill_sch_appt AS a
INNER JOIN 4_prod.raw.mill_sch_event AS e
ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
INNER JOIN 4_prod.raw.mill_person AS p
ON a.PERSON_ID = p.PERSON_ID
WHERE 
  APPT_SYNONYM_FREE ILIKE '%antenatal%'
  AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
  AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
LIMIT 100

In [0]:
%sql
-- Number of adult women who received antenatal care between 2019-12-01 and 2024-12-31 and were recorded with at least one 'CMV' clinical event
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
WHERE
  EVENT_TITLE_TEXT ILIKE '%CMV%'
  --AND EVENT_START_DT_TM BETWEEN '2019-12-01' AND '2024-12-31' -- got 0 result


In [0]:
%sql
-- Number of adult women who received antenatal care between 2019-12-01 and 2024-12-31 and were recorded with at least one clinical event with CMV-related EVENT_CD in the same period
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- linked with a (PI_CDE) blob content that contains CMV keywords
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
ce AS (
  SELECT EVENT_ID, MAX(PERSON_ID) AS PERSON_ID
  FROM 4_prod.raw.mill_clinical_event
  WHERE COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
  GROUP BY EVENT_ID
),
b AS (
  SELECT bc.EVENT_ID, BLOB_CONTENTS, CE_BLOB_CONTENT_ID, ante.PERSON_ID
  FROM 4_prod.raw.pi_cde_blob_content AS bc
  INNER JOIN ce
  ON bc.EVENT_ID = ce.EVENT_ID
  INNER JOIN ante
  ON ante.PERSON_ID = ce.PERSON_ID
  WHERE BLOB_CONTENTS ILIKE '%CMV%' OR BLOB_CONTENTS ILIKE '%Cytomegalovirus%'
)
SELECT COUNT(*), COUNT(DISTINCT b.PERSON_ID), COUNT(DISTINCT b.CE_BLOB_CONTENT_ID)
FROM b


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- linked with a (PI_CDE) blob content that contains CMV keywords
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
ce AS (
  SELECT EVENT_ID, MAX(PERSON_ID) AS PERSON_ID
  FROM 4_prod.raw.mill_clinical_event
  WHERE COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
  GROUP BY EVENT_ID
),
b AS (
  SELECT bc.EVENT_ID, BLOB_CONTENTS, CE_BLOB_CONTENT_ID, ante.PERSON_ID
  FROM 4_prod.raw.pi_cde_blob_content AS bc
  INNER JOIN ce
  ON bc.EVENT_ID = ce.EVENT_ID
  INNER JOIN ante
  ON ante.PERSON_ID = ce.PERSON_ID
  WHERE BLOB_CONTENTS ILIKE '%CMV%' OR BLOB_CONTENTS ILIKE '%Cytomegalovirus%'
)
SELECT *
FROM b
LIMIT 100

In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- linked with a (PI_CDE) blob content
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID), COUNT(DISTINCT bc.CE_BLOB_CONTENT_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.pi_cde_blob_content AS bc
ON ce.EVENT_ID = bc.EVENT_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- clinical event linked with a (PI_CDE) blob content
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT ce.PERSON_ID), COUNT(DISTINCT bc.CE_BLOB_CONTENT_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN ante
ON ce.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.pi_cde_blob_content AS bc
ON ce.EVENT_ID = bc.EVENT_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'


In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN ante
ON bc.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql
/* 
Number of distinct patients who are
  -- new born babies
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(DAY, BIRTH_DT_TM, BEG_DT_TM) <= 28
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN ante
ON bc.PERSON_ID = ante.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql

-- Number of new born babies recorded with antenatal care appointments

SELECT COUNT(DISTINCT a.PERSON_ID)
FROM 4_prod.raw.mill_sch_appt AS a
INNER JOIN 4_prod.raw.mill_sch_event AS e
ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
INNER JOIN 4_prod.raw.mill_person AS p
ON a.PERSON_ID = p.PERSON_ID
WHERE 
  APPT_SYNONYM_FREE ILIKE '%antenatal%'
  AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
  AND DATEDIFF(DAY, BIRTH_DT_TM, BEG_DT_TM) <= 28
  LIMIT 100

In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
cmv_ce AS (
  SELECT ce.PERSON_ID
  FROM 4_prod.raw.mill_clinical_event AS ce
  INNER JOIN 3_lookup.mill.mill_code_value AS cv
  ON ce.EVENT_CD = cv.CODE_VALUE
  INNER JOIN ante
  ON ce.PERSON_ID = ante.PERSON_ID
  WHERE
    (cv.DESCRIPTION LIKE '%CMV%' OR cv.DESCRIPTION ILIKE '%Cytomegalovirus%')
    AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM (SELECT * FROM 4_prod.rde.rde_blobdataset TIMESTAMP AS OF '2025-08-26') AS bc
INNER JOIN cmv_ce
ON bc.PERSON_ID = cmv_ce.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql
/* 
Number of distinct patients who are
  -- female
  -- age 18+
  -- received antenatal care between 2019-12-01 and 2024-12-31
  -- [removed] recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
cmv_ce AS (
  SELECT ce.PERSON_ID
  FROM 4_prod.raw.mill_clinical_event AS ce
  INNER JOIN 3_lookup.mill.mill_code_value AS cv
  ON ce.EVENT_CD = cv.CODE_VALUE
  INNER JOIN ante
  ON ce.PERSON_ID = ante.PERSON_ID
  WHERE
    (cv.DESCRIPTION LIKE '%CMV%' OR cv.DESCRIPTION ILIKE '%Cytomegalovirus%')
    AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
--INNER JOIN cmv_ce
--ON bc.PERSON_ID = cmv_ce.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql

-- Number of adult women who were recorded at least one clinical event with EVENT_CD related to CMV regardless of attending any antenatal appointment
SELECT COUNT(DISTINCT ce.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 4_prod.raw.mill_person AS p
ON ce.PERSON_ID = p.PERSON_ID
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
WHERE 
  cv.DESCRIPTION LIKE '%CMV%'
  AND SEX_CD = 362 -- female
  AND DATEDIFF(YEAR, BIRTH_DT_TM, EVENT_START_DT_TM) >= 18
  AND COALESCE(EVENT_END_DT_TM, EVENT_START_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'

In [0]:
%sql

WITH cohort AS (
  SELECT DISTINCT a.PERSON_ID FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND BIRTH_DT_TM < '2001-12-01'
),
ce AS (
  SELECT EVENT_ID, MAX(PERSON_ID) AS PERSON_ID
  FROM 4_prod.raw.mill_clinical_event
  GROUP BY EVENT_ID
)
SELECT b.* 
FROM 4_prod.raw.pi_cde_blob_content AS b
INNER JOIN ce
ON ce.EVENT_ID = b.EVENT_ID
INNER JOIN cohort
ON cohort.PERSON_ID = ce.PERSON_ID
WHERE BLOB_CONTENTS LIKE '%CMV%'

LIMIT 100


In [0]:
%sql

SELECT COUNT(DISTINCT p.PERSON_ID)
FROM 4_prod.raw.mill_clinical_event AS e
INNER JOIN 4_prod.raw.mill_person AS p
ON e.PERSON_ID = p.PERSON_ID
WHERE 
  EVENT_TITLE_TEXT LIKE '%CMV%'
  AND DATEDIFF(DAY, p.BIRTH_DT_TM, e.EVENT_START_DT_TM) <= 28 

LIMIT 100


In [0]:
%sql
/* 
Number of distinct patients who are
  -- new born <= 28 days
  -- recorded with 1+ clinical event with CMV-related EVENT_CD in the same period
  -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH cmv_ce AS (
  SELECT ce.PERSON_ID
  FROM 4_prod.raw.mill_clinical_event AS ce
  INNER JOIN 3_lookup.mill.mill_code_value AS cv
  ON ce.EVENT_CD = cv.CODE_VALUE
  INNER JOIN 4_prod.raw.mill_person AS p
  ON p.PERSON_ID = ce.PERSON_ID
  WHERE
    cv.DESCRIPTION LIKE '%CMV%'
    AND DATEDIFF(DAY, p.BIRTH_DT_TM, COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM)) <= 28 
    AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT COUNT(DISTINCT bc.PERSON_ID), COUNT(DISTINCT bc.EVENTID)
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN cmv_ce
ON bc.PERSON_ID = cmv_ce.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
LIMIT 100



In [0]:
%sql

/* 
Number of distinct new born patients
  -- who are 28 days or less when a CMV clinical event happened 
  -- at least 1 CMV-related clinical event happened between 2019-12-01 and 2024-12-31
  -- whose mothers are
        -- recorded as Mother in Mill_PERSON_PERSON_RELTN
        -- age 18+
        -- received antenatal care between 2019-12-01 and 2024-12-31
        -- recorded with at least 1 clinical event related to CMV in the same period
        -- person_id linked to at least 1 RDE blobdataset in the same period
*/
WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
cmv_ce AS (
  SELECT ce.PERSON_ID
  FROM 4_prod.raw.mill_clinical_event AS ce
  INNER JOIN 3_lookup.mill.mill_code_value AS cv
  ON ce.EVENT_CD = cv.CODE_VALUE
  INNER JOIN ante
  ON ce.PERSON_ID = ante.PERSON_ID
  WHERE
    (cv.DESCRIPTION LIKE '%CMV%' OR cv.DESCRIPTION ILIKE '%Cytomegalovirus%')
    AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
),
cmv_mother AS (
SELECT DISTINCT bc.PERSON_ID
FROM 4_prod.rde.rde_blobdataset AS bc
INNER JOIN cmv_ce
ON bc.PERSON_ID = cmv_ce.PERSON_ID
INNER JOIN 4_prod.raw.mill_clinical_event AS ce
ON ce.EVENT_ID = bc.EVENTID 
WHERE 
  (blobcontents LIKE '%CMV%' OR blobcontents ILIKE '%Cytomegalovirus%')
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
),
cmv_child AS (
SELECT DISTINCT ce.PERSON_ID
FROM 4_prod.raw.mill_clinical_event AS ce
INNER JOIN 3_lookup.mill.mill_code_value AS cv
ON ce.EVENT_CD = cv.CODE_VALUE
INNER JOIN 4_prod.raw.mill_person AS p
ON p.PERSON_ID = ce.PERSON_ID
WHERE
  cv.DESCRIPTION LIKE '%CMV%'
  AND DATEDIFF(DAY, p.BIRTH_DT_TM, COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM)) <= 28 
  AND COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM, ce.VALID_FROM_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
)
SELECT COUNT(DISTINCT r.PERSON_ID)  
FROM 4_prod.raw.mill_person_person_reltn AS r
LEFT JOIN 3_lookup.mill.mill_code_value AS cd
ON r.PERSON_RELTN_CD = cd.CODE_VALUE
INNER JOIN cmv_child
ON cmv_child.PERSON_ID = r.PERSON_ID
INNER JOIN cmv_mother
ON r.RELATED_PERSON_ID = cmv_mother.PERSON_ID
WHERE
  DESCRIPTION = 'Mother' -- mother-child relation

In [0]:
%sql

SELECT * FROM 4_prod.raw.mill_person WHERE PERSON_ID = 54118028

In [0]:
%sql

SELECT * FROM 4_prod.raw.mill_person
WHERE BIRTH_DT_TM > '2025-07-01'
LIMIT 10

In [0]:
%sql

SELECT * FROM 4_prod.raw.mill_clinical_event
WHERE EVENT_TITLE_TEXT LIKE '%CMV%'
LIMIT 1000

In [0]:
%sql

SELECT DISTINCT TFCCode, TFCDesc_Full
FROM 4_prod.raw.path_master_resultable
WHERE TFCDesc_Full ILIKE '%CMV%'
-- OR TFCDesc_Full ILIKE '%Cytomegalovirus%' -- No result with full name

In [0]:
%sql
WITH tfc AS (
  SELECT DISTINCT TFCCode
  FROM 4_prod.raw.path_master_resultable
  WHERE TFCDesc_Full ILIKE '%CMV%'
)
SELECT *
FROM 4_prod.raw.path_patient_resultlevel AS r
INNER JOIN tfc
ON r.TFCCode = tfc.TFCCode


In [0]:
%sql
-- These are probably the composite keys
SELECT LIMSNo, LabNo, LegWkgCode, COUNT(*)
FROM 4_prod.raw.path_patient_samplelevel
GROUP BY LIMSNo, LabNo, LegWkgCode
HAVING COUNT(*) > 1


In [0]:
%sql
SELECT *
FROM 4_prod.raw.path_patient_samplelevel

In [0]:
%sql

WITH ante AS (
  SELECT DISTINCT a.PERSON_ID
  FROM 4_prod.raw.mill_sch_appt AS a
  INNER JOIN 4_prod.raw.mill_sch_event AS e
  ON a.SCH_EVENT_ID = e.SCH_EVENT_ID
  INNER JOIN 4_prod.raw.mill_person AS p
  ON a.PERSON_ID = p.PERSON_ID
  WHERE 
    APPT_SYNONYM_FREE ILIKE '%antenatal%'
    AND BEG_DT_TM BETWEEN '2019-12-01' AND '2024-12-31'
    AND DATEDIFF(YEAR, BIRTH_DT_TM, BEG_DT_TM) >= 18
),
ce AS (
  SELECT EVENT_ID, MAX(PERSON_ID) AS PERSON_ID
  FROM 4_prod.raw.mill_clinical_event
  WHERE COALESCE(EVENT_START_DT_TM, EVENT_END_DT_TM) BETWEEN '2019-12-01' AND '2024-12-31'
  GROUP BY EVENT_ID
),
b AS (
  SELECT bc.EVENT_ID, BLOB_CONTENTS, CE_BLOB_CONTENT_ID, ante.PERSON_ID
  FROM 4_prod.raw.pi_cde_blob_content AS bc
  INNER JOIN ce
  ON bc.EVENT_ID = ce.EVENT_ID
  INNER JOIN ante
  ON ante.PERSON_ID = ce.PERSON_ID
  WHERE BLOB_CONTENTS ILIKE '%CMV%' OR BLOB_CONTENTS ILIKE '%Cytomegalovirus%'
),
tfc AS (
  SELECT DISTINCT TFCCode
  FROM 4_prod.raw.path_master_resultable
  WHERE TFCDesc_Full ILIKE '%CMV%'
),
mrn AS (
    SELECT
        MAX(PERSON_ID) AS person_id,
        ALIAS AS MRN
    FROM 4_prod.raw.MILL_PERSON_ALIAS 
    WHERE PERSON_ALIAS_TYPE_CD = 10 -- MRN
    GROUP BY ALIAS
),
nhs AS (
    SELECT
      MAX(PERSON_ID) AS person_id,
      ALIAS AS nhs_nbr
    FROM 4_prod.rde.patient_nhs
    GROUP BY ALIAS
),
sam AS (
  SELECT LIMSNo, LabNo, LegWkgCode, COALESCE(mrn.person_id, nhs.person_id) AS PERSON_ID
  FROM 4_prod.raw.path_patient_samplelevel AS s
  LEFT JOIN mrn
  ON s.MRN = mrn.MRN
  LEFT JOIN nhs
  ON s.NHSNo = nhs.nhs_nbr
  WHERE RequestDate BETWEEN '2023-12-01' AND '2024-12-31'
),
result AS (
  SELECT r.*, sam.PERSON_ID
  FROM 4_prod.raw.path_patient_resultlevel AS r
  INNER JOIN tfc
  ON r.TFCCode = tfc.TFCCode
  INNER JOIN sam
  ON sam.LIMSNo = r.LIMSNo AND sam.LabNo = r.LabNo AND sam.LegWkgCode = r.LegWkgCode
  INNER JOIN b
  ON sam.PERSON_ID = b.PERSON_ID
)
SELECT PERSON_ID, COUNT(*)
FROM result
GROUP BY PERSON_ID
